https://mset.space - платформа для проведения анализа данных, построения моделей и внедрения моделей в продукцию.


https://mset.space - platform for data analysis, model building and implementation into production.


Ноутбуки для платформы mset.space
https://github.com/MindSetLib/MS-Education

Low code ML - библиотека машинного обучения
https://github.com/MindSetLib/Insolver


Телеграм канал платформы
https://t.me/msetspace

Связь с разработчиками:
telegram: @frankshikhaliev
email: request@mind-set.ru

# Optuna
### Установка

Библиотека [Optuna](https://optuna.org/) создана для автоматического поиска оптимальных гиперпараметров моделей. 

Посмотрим, установлена ли библиотека. Если установлена, то в output ячейки будет написана версия. В нашем случае библиотека не установлена.

In [ ]:
#freeze: отобразит список установленных библиотек
#| grep: найдёт указаное после название и выведет в output
!pip freeze | grep optuna

Установим библиотеку Optuna.

In [ ]:
# -q: флажек, чтобы не отображать подробную информацию в output
!pip install optuna -q

     |████████████████████████████████| 266kB 5.9MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 30.6MB/s 
     |████████████████████████████████| 81kB 5.0MB/s 
     |████████████████████████████████| 81kB 8.2MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 133kB 25.3MB/s 
     |████████████████████████████████| 112kB 27.0MB/s 


Проверим снова.

In [ ]:
!pip freeze | grep optuna

optuna==2.3.0


Библиотека установлена, можно приступать к работе.

### Оптимизация целевой функции

In [ ]:
import joblib
import optuna

import pandas as pd

Обычно библиотека Optuna используется для автоматического поиска гиперпараметров, но чтобы разобрать концепцию библиотеки, мы будем оптимизируем квадратичную функцию $(x - 2)^2$.

Функцию, которую планируем оптимизировать, будем называть **целевой функцией (objective)**.

Наша задача найти $x$, при котором целевая функция $(x - 2)^2$ достигает минимума. Для этого нам необходимо определить функцию `objective`, в качестве аргумента передать ей объект `trial` и вернуть значение целевой функции. `trial` модуль можно использовать для получения гиперпараметров. Функция `objective` вызывается библиотекой `Optuna` много раз, чтобы найти оптимальные параметры.

In [ ]:
def objective(trial: optuna.Trial):
    x = trial.suggest_uniform('x', -10, 10) # укажем диапазон значений для x 
    return (x - 2) ** 2                     # вернем значение целевой функции

Используя модуль `trial` можно указать пространство параметров, следующиx видов:

In [ ]:
# Categorical parameter
optimizer = trial.suggest_categorical('optimizer', ['MomentumSGD', 'Adam']) # вначале указываем название переменной 'optimizer', 
                                                                            # после указываем диапазон значений
# Int parameter
num_layers = trial.suggest_int('num_layers', 1, 3)

# Uniform parameter
dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 1.0)

# Loguniform parameter
learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)

# Discrete-uniform parameter
drop_path_rate = trial.suggest_discrete_uniform('drop_path_rate', 0.0, 1.0, 0.1)

Чтобы начать оптимизацию, нам достаточно записать всего 2 строчки кода!

In [ ]:
study = optuna.create_study()           # optuna.create_study(direction = 'maximize') если хотим максимизировать функцию
study.optimize(objective, n_trials=10)

[I 2020-12-01 17:46:01,755] A new study created in memory with name: no-name-26d947eb-1c2b-42f2-b7ad-fe06bce4a253
[I 2020-12-01 17:46:01,758] Trial 0 finished with value: 2.1345192992690367 and parameters: {'x': 3.460999417956433}. Best is trial 0 with value: 2.1345192992690367.
[I 2020-12-01 17:46:01,763] Trial 1 finished with value: 45.46167089849976 and parameters: {'x': -4.742527040991364}. Best is trial 0 with value: 2.1345192992690367.
[I 2020-12-01 17:46:01,767] Trial 2 finished with value: 29.193698914064246 and parameters: {'x': 7.403119368852057}. Best is trial 0 with value: 2.1345192992690367.
[I 2020-12-01 17:46:01,769] Trial 3 finished with value: 21.583477659413884 and parameters: {'x': 6.645802154570713}. Best is trial 0 with value: 2.1345192992690367.
[I 2020-12-01 17:46:01,775] Trial 4 finished with value: 111.01984012795677 and parameters: {'x': -8.536595281586779}. Best is trial 0 with value: 2.1345192992690367.
[I 2020-12-01 17:46:01,778] Trial 5 finished with value

Целевую функцию можно не только минимизировать, но и оптимизировать, для этого необходимо указать параметр `direction = 'maximize'`

In [ ]:
optuna.create_study(direction = 'maximize')

Можно эксперименту задать уникальное имя

In [ ]:
optuna.create_study(study_name = 'unique_name')

Узнать имя эксперимента

In [ ]:
study.study_name

'no-name-26d947eb-1c2b-42f2-b7ad-fe06bce4a253'

Получить лучшие параметры

In [ ]:
study.best_params

{'x': 1.5499045851321895}

Получить лучшее минимальное значение целевой функции

In [ ]:
study.best_value

0.2025858824850264

Получить параметры лучшего запуска `trial`

In [ ]:
study.best_trial

FrozenTrial(number=8, value=0.2025858824850264, datetime_start=datetime.datetime(2020, 12, 1, 17, 46, 1, 781798), datetime_complete=datetime.datetime(2020, 12, 1, 17, 46, 1, 781994), params={'x': 1.5499045851321895}, distributions={'x': UniformDistribution(high=10, low=-10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=8, state=TrialState.COMPLETE)

Запустив метод `optimize` мы можем продолжить оптимизацию

In [ ]:
study.optimize(objective, n_trials=5)

[I 2020-12-01 17:50:15,110] Trial 10 finished with value: 55.89508419978993 and parameters: {'x': 9.476301505409605}. Best is trial 8 with value: 0.2025858824850264.
[I 2020-12-01 17:50:15,114] Trial 11 finished with value: 0.7376560671149242 and parameters: {'x': 2.8588690628465576}. Best is trial 8 with value: 0.2025858824850264.
[I 2020-12-01 17:50:15,119] Trial 12 finished with value: 2.456642730693781 and parameters: {'x': 3.567368090364794}. Best is trial 8 with value: 0.2025858824850264.
[I 2020-12-01 17:50:15,123] Trial 13 finished with value: 1.4596894468946828 and parameters: {'x': 3.208176082735742}. Best is trial 8 with value: 0.2025858824850264.
[I 2020-12-01 17:50:15,127] Trial 14 finished with value: 36.74479192914257 and parameters: {'x': -4.061748256826785}. Best is trial 8 with value: 0.2025858824850264.


In [ ]:
study.best_params

{'x': 1.5499045851321895}

### Сохранение и загрузка истории

Бывает в ходе эксперимента, мы добавляем или убираем какие-то переменные. Хотелось бы сохранить историю тренировок, чтобы потом её можно было востановить. Сделать эт можно несколькими способами.

Можно получить DataFrame отчёт об оптимизации и сохранить его.

In [ ]:
df = study.trials_dataframe()
df

,number,value,datetime_start,datetime_complete,duration,params_x,state
0,0,2.134519,2020-12-01 17:46:01.757983,2020-12-01 17:46:01.758489,0 days 00:00:00.000506,3.460999,COMPLETE
1,1,45.461671,2020-12-01 17:46:01.762345,2020-12-01 17:46:01.762665,0 days 00:00:00.000320,-4.742527,COMPLETE
2,2,29.193699,2020-12-01 17:46:01.767178,2020-12-01 17:46:01.767528,0 days 00:00:00.000350,7.403119,COMPLETE
3,3,21.583478,2020-12-01 17:46:01.768742,2020-12-01 17:46:01.768944,0 days 00:00:00.000202,6.645802,COMPLETE
4,4,111.019840,2020-12-01 17:46:01.774748,2020-12-01 17:46:01.774977,0 days 00:00:00.000229,-8.536595,COMPLETE
5,5,10.087298,2020-12-01 17:46:01.777761,2020-12-01 17:46:01.777979,0 days 00:00:00.000218,-1.176051,COMPLETE
6,6,51.539216,2020-12-01 17:46:01.778959,2020-12-01 17:46:01.779125,0 days 00:00:00.000166,-5.179082,COMPLETE
7,7,6.991018,2020-12-01 17:46:01.780062,2020-12-01 17:46:01.780202,0 days 00:00:00.000140,-0.644053,COMPLETE
8,8,0.202586,2020-12-01 17:46:01.781798,2020-12-01 17:46:01.781994,0 days 00:00:00.000196,1.549905,COMPLETE
9,9,1.080682,2020-12-01 17:46:01.783306,2020-12-01 17:46:01.783469,0 days 00:00:00.000163,0.960441,COMPLETE


In [ ]:
df.to_csv('study.csv') # сохраним в файл

In [ ]:
loaded = pd.read_csv('study.csv') # загрузим файл в переменную loaded
loaded

,Unnamed: 0,number,value,datetime_start,datetime_complete,duration,params_x,state
0,0,0,2.134519,2020-12-01 17:46:01.757983,2020-12-01 17:46:01.758489,0 days 00:00:00.000506,3.460999,COMPLETE
1,1,1,45.461671,2020-12-01 17:46:01.762345,2020-12-01 17:46:01.762665,0 days 00:00:00.000320,-4.742527,COMPLETE
2,2,2,29.193699,2020-12-01 17:46:01.767178,2020-12-01 17:46:01.767528,0 days 00:00:00.000350,7.403119,COMPLETE
3,3,3,21.583478,2020-12-01 17:46:01.768742,2020-12-01 17:46:01.768944,0 days 00:00:00.000202,6.645802,COMPLETE
4,4,4,111.019840,2020-12-01 17:46:01.774748,2020-12-01 17:46:01.774977,0 days 00:00:00.000229,-8.536595,COMPLETE
5,5,5,10.087298,2020-12-01 17:46:01.777761,2020-12-01 17:46:01.777979,0 days 00:00:00.000218,-1.176051,COMPLETE
6,6,6,51.539216,2020-12-01 17:46:01.778959,2020-12-01 17:46:01.779125,0 days 00:00:00.000166,-5.179082,COMPLETE
7,7,7,6.991018,2020-12-01 17:46:01.780062,2020-12-01 17:46:01.780202,0 days 00:00:00.000140,-0.644053,COMPLETE
8,8,8,0.202586,2020-12-01 17:46:01.781798,2020-12-01 17:46:01.781994,0 days 00:00:00.000196,1.549905,COMPLETE
9,9,9,1.080682,2020-12-01 17:46:01.783306,2020-12-01 17:46:01.783469,0 days 00:00:00.000163,0.960441,COMPLETE


Можно сделать дамп самого оптимизатора

In [ ]:
joblib.dump(study, 'experiments.pkl')  # сохраним в формате .pkl

['experiments.pkl']

In [ ]:
study_loaded = joblib.load('experiments.pkl') # загрузим оптимизатор в переменную study_loaded

In [ ]:
study_loaded.trials_dataframe() # откроем историю, чтобы убедится, что это наш оптимизатор

,number,value,datetime_start,datetime_complete,duration,params_x,state
0,0,2.134519,2020-12-01 17:46:01.757983,2020-12-01 17:46:01.758489,0 days 00:00:00.000506,3.460999,COMPLETE
1,1,45.461671,2020-12-01 17:46:01.762345,2020-12-01 17:46:01.762665,0 days 00:00:00.000320,-4.742527,COMPLETE
2,2,29.193699,2020-12-01 17:46:01.767178,2020-12-01 17:46:01.767528,0 days 00:00:00.000350,7.403119,COMPLETE
3,3,21.583478,2020-12-01 17:46:01.768742,2020-12-01 17:46:01.768944,0 days 00:00:00.000202,6.645802,COMPLETE
4,4,111.019840,2020-12-01 17:46:01.774748,2020-12-01 17:46:01.774977,0 days 00:00:00.000229,-8.536595,COMPLETE
5,5,10.087298,2020-12-01 17:46:01.777761,2020-12-01 17:46:01.777979,0 days 00:00:00.000218,-1.176051,COMPLETE
6,6,51.539216,2020-12-01 17:46:01.778959,2020-12-01 17:46:01.779125,0 days 00:00:00.000166,-5.179082,COMPLETE
7,7,6.991018,2020-12-01 17:46:01.780062,2020-12-01 17:46:01.780202,0 days 00:00:00.000140,-0.644053,COMPLETE
8,8,0.202586,2020-12-01 17:46:01.781798,2020-12-01 17:46:01.781994,0 days 00:00:00.000196,1.549905,COMPLETE
9,9,1.080682,2020-12-01 17:46:01.783306,2020-12-01 17:46:01.783469,0 days 00:00:00.000163,0.960441,COMPLETE


### Визуализация

**История оптимизации**

Синяя точка — значение целевой функции с сэмплированным параметром.

Красная точка — значение целевой функции, с лучшим параметром найденым в момент $t$.

In [ ]:
optuna.visualization.plot_optimization_history(study)

**История поиска оптимального параметра**

Белые точки показывают рание стадии, синии последнии стадии поиска параметров. Видим, как постепенно `Optuna` "спускается" к минимуму целевой функции.

In [ ]:
optuna.visualization.plot_slice(study)

Оставшиеся визуализации для оптимизации с несколькими параметрами, чтобы их построить вначале оптимизируем целевую функцию $(x + y - 2)^2$ по параметрам $x$ и $y$.

In [ ]:
def objective(trial: optuna.Trial):
    x = trial.suggest_uniform('x', -10, 10)
    y = trial.suggest_uniform('y', -10, 10)
    return (x + y - 2) ** 2 

In [ ]:
study = optuna.create_study()
study.optimize(objective, n_trials=40)

[I 2020-12-01 17:56:24,299] A new study created in memory with name: no-name-05a5e65d-339c-494e-8f56-5bfb1a9a7170
[I 2020-12-01 17:56:24,305] Trial 0 finished with value: 58.82051622560095 and parameters: {'x': 2.3361390291661763, 'y': 7.333314421087039}. Best is trial 0 with value: 58.82051622560095.
[I 2020-12-01 17:56:24,310] Trial 1 finished with value: 3.0261284209365362 and parameters: {'x': 3.8456266418756506, 'y': -3.585203722932002}. Best is trial 1 with value: 3.0261284209365362.
[I 2020-12-01 17:56:24,316] Trial 2 finished with value: 3.067717044135488 and parameters: {'x': 1.130750665884289, 'y': -0.882240615651483}. Best is trial 1 with value: 3.0261284209365362.
[I 2020-12-01 17:56:24,322] Trial 3 finished with value: 296.19120366831515 and parameters: {'x': -8.342844626271189, 'y': -6.8673617544877334}. Best is trial 1 with value: 3.0261284209365362.
[I 2020-12-01 17:56:24,324] Trial 4 finished with value: 66.2255625876315 and parameters: {'x': -6.4706393436632865, 'y': 

История поиска 

In [ ]:
optuna.visualization.plot_contour(study)

По данному графику можно посмотреть, в совокупности (какие значения каких параметров дают лучший результат), какие параметры лучше, чем синее тем линия лучше.

In [ ]:
optuna.visualization.plot_parallel_coordinate(study)

### Pruning

Можно выделить 3 основных метода подборов параметров:

*  Решетка. Перебор всех возможных комбинаций параметров.

*  Случайный поиск. Случайным образом выбираем комбинацию параметров.

*  Адаптивный отбор. В библиотеке Optuna используются алгоритмы median stopping rule и Successive Halving (SHA).

![](https://s8.hostingkartinok.com/uploads/images/2020/11/019a65a57f303d9506769fcb99c99621.jpeg)

В библиотеке Optuna есть механизм под названием Pruning, простыми словами раняя остановка вычислений бесперспективных комбинаций параметров.

Рассмотрим pruning с алгоритмом SHA. Допустим, мы хотели бы оптимизировать долю верных ответов (accuracy) используя три параметра в каждом из которых два возможных значения. Например, `max_depth = [30, 50]`, `n_estimater = [500, 1000]`, `criterion = ['gini', 'entropy']`.

Всего мы имеем $2^3 = 8$ комбинаций параметров. Давайте запустим все и замерим accuracy.

![](https://s8.hostingkartinok.com/uploads/images/2020/11/693ec1bfcb8ebdea815b56be6c1a8557.png)

 После чего откинем половину комбинаций, показывающих худшие результаты и протестируем оставшиеся. 

![](https://s8.hostingkartinok.com/uploads/images/2020/11/6b033dcce7d3247389e7f420bc47fa99.png)

Так будем продолжать иттерационно, пока не найдём оптимальную комбинацию.

Мы ограничены в вычислительных ресурсах, поэтому алгоритм SHA имеет свой гиперпараметр, обозначим его $n$, количество комбинаций параметров.

По построению алгоритма, с увеличеснием $n$, нам нужно выделять больше количество эпох, зависимое как $log_2(n)$.

Чем больше $n$, тем потенциально более нестабильное поведение может быть у кривой accuracy, т.е. могут быть какие-то неравномерные возрастания, что может привести к тому, что мы отбросим перспективную ветвь. Обратим внимание на синюю ветку на графике, если бы у нас было количество параметров равное 6, то мы бы после первой эпохи откинули её следуя алгоритму SHA, но как видим она имеет хороший потенциал.

Именно эта проблема исправлена в алгоритме [Hyperband](https://medium.com/optuna/optuna-supports-hyperband-93b0cae1a137), который на данный момент 25.11.2020 является лучшим.


### Пример Pruning

В одном примере разберем сразу:

*  как можно оптимизировать параметры модели

*  как оптимизировать параметры k моделей

*  как использовать Pruning



Список алгоритмов pruning в Optuna: [алгоритмы](https://optuna.readthedocs.io/en/stable/reference/pruners.html)

По умолчанию стоит `MedianPruner` (информации [отсюда](https://medium.com/optuna/optuna-v2-3165e3f1fc2))




In [ ]:
# как установить алгоритм Pruning
study = optuna.create_study(pruner=optuna.pruners.HyperbandPruner(max_resource='auto'))

Список сэмплеров параметров: [сэмплеры](https://optuna.readthedocs.io/en/stable/reference/samplers.html)

Более продвинутые сэмплеры используют информацию о том, какие ветки были обрезаны.

На данный момент 25.11.2020 лучший сэмплер `CmaEsSampler`.

In [ ]:
# как установить сэмплер
study = optuna.create_study(sampler=optuna.integration.CmaEsSampler())

In [ ]:
# или так
study = optuna.create_study(sampler=optuna.samplers.CmaEsSampler())

In [ ]:
import optuna
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler

# Загрузим датасет из Sklearn
X, y = datasets.load_breast_cancer(return_X_y = True)

# Нормируем данные
scaler = StandardScaler()
X = scaler.fit_transform(X)

x_train, x_valid, y_train, y_valid = model_selection.train_test_split(X, y)

#Step 1. Объявим целевую функцию для её максимизации
def objective(trial):

    cls_name = trial.suggest_categorical('classifier', ['LogReg', 'RandomForest'])
    
    # Step 2. Установим значения для гиперпараметров:
    if cls_name == 'LogReg':
        # установим параметры, которые будет оптимизировать
        # коэф. C = 1/λ
        C = trial.suggest_float('C', 1e-10, 1, log=True)
        # инициализируем логистическую регрессию
        cls = LogisticRegression(C = C)
    else:
        # установим параметры, которые будет оптимизировать
        n_estimators = trial.suggest_int('n_estimators', 10, 1000)
        max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
        # инициализируем решающие деревья
        cls = RandomForestClassifier(max_depth = max_depth,
                                     n_estimators = n_estimators)

    # Step 3. Прерываем обучение, если оно не эффективно
    for step in range(100):
        cls.fit(x_train, y_train)

        # classifier_obj.score: возвращает метрику r^2
        intermediate_value = cls.score(x_valid, y_valid)
        # укажем значение целевой функция на данном шаге
        trial.report(intermediate_value, step)

        # Принимаем решение, стоит ли прерывать дальнейшие вычисления
        if trial.should_prune():
            # raise: вызывает ошибку, которая не входит в стандартный язык python https://stackoverflow.com/questions/13957829/how-to-use-raise-keyword-in-python
            # optuna.TrialPruned(): сообщает текущему trial, что вычисления были прерваны https://optuna.readthedocs.io/en/stable/reference/generated/optuna.TrialPruned.html
            raise optuna.TrialPruned()

        return intermediate_value

# Step 4. Запустим оптимизацию
study = optuna.create_study(direction = 'maximize') # использовать другие pruning: pruner = optuna.pruners.HyperbandPruner(max_resource = 'auto')
study.optimize(objective, n_trials = 100)

[I 2020-12-01 18:12:13,392] A new study created in memory with name: no-name-7912b42b-c61e-44c4-9628-6d8ad697d1be
[I 2020-12-01 18:12:13,424] Trial 0 finished with value: 0.6643356643356644 and parameters: {'classifier': 'LogReg', 'C': 3.0698730423284656e-09}. Best is trial 0 with value: 0.6643356643356644.
[I 2020-12-01 18:12:13,945] Trial 1 finished with value: 0.972027972027972 and parameters: {'classifier': 'RandomForest', 'n_estimators': 252, 'max_depth': 3}. Best is trial 1 with value: 0.972027972027972.
[I 2020-12-01 18:12:15,031] Trial 2 finished with value: 0.972027972027972 and parameters: {'classifier': 'RandomForest', 'n_estimators': 561, 'max_depth': 12}. Best is trial 1 with value: 0.972027972027972.
[I 2020-12-01 18:12:15,041] Trial 3 finished with value: 0.6643356643356644 and parameters: {'classifier': 'LogReg', 'C': 1.281938246444521e-09}. Best is trial 1 with value: 0.972027972027972.
[I 2020-12-01 18:12:16,555] Trial 4 finished with value: 0.9790209790209791 and par

In [ ]:
# Посчитаем кол-во прерванных и кол-во завершенных trials
pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print('  Number of finished trials: ', len(study.trials))
print('  Number of pruned trials: ', len(pruned_trials))
print('  Number of complete trials: ', len(complete_trials))

  Number of finished trials:  100
  Number of pruned trials:  32
  Number of complete trials:  68


Посмотрим параметры с которыми получили лучшие результаты

In [ ]:
study.best_params

{'C': 0.1511216155188127, 'classifier': 'LogReg'}

На практике, небольшая доля параметров вносит значительный вклад в улучшение модели, остальные же параметры не важные.

С добавлением нового параметра для оптимизации, вычислительная сложность возрастает экспоненциально. Поэтому очень важно знать, какие параметры не вносят вклад в увеличение качества модели и не использовать их при оптимизации.

Можно получить список важности параметров, визуализировать их и проводить оптимизацию по самым важным, чтобы сократить вычисления.

In [ ]:
# получим список важности переменных
importances = optuna.importance.get_param_importances(study)
# визуализируем данные
fig = optuna.visualization.plot_param_importances(study)
fig.show()

Как выглядит визуализация, когда параметров много. Все параметры в сумме дают единицу.

![](https://s8.hostingkartinok.com/uploads/images/2020/11/a5a6f7d549ff943e57cc7db15a91ec01.png)

### Интеграции

Optuna гибкая система, с её помощью можно оптимизировать любую функцию, любой алгоритм. Несмотря на это, данная библиотека плотно сотрудничает с другими библиотеками для машинного и глубокого обучения.

Есть множество [интеграций](https://optuna.readthedocs.io/en/stable/reference/integration.html#), которые упрощают оптимизацию различных моделей. Простота заключается в том, что необходимо написать всего одну строчку кода, чтобы использовать оптимизатор из Optuna.

Давайте рассмотрим на [примере библиотеки LightGBM](https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258).

In [ ]:
import numpy as np
import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# всё, что нам нужно сделать, чтобы использовать LightGBM + Optuna, это импорт этого модуля
import optuna.integration.lightgbm as lgb

# загрузим данные
data, target = sklearn.datasets.load_breast_cancer(return_X_y = True)
# разделим на тестовую и обучающую выборку
train_x, val_x, train_y, val_y = train_test_split(data, target, test_size=0.25)
# сформируем данные для подачи в LightGBM
dtrain, dval = lgb.Dataset(train_x, label=train_y, params={'verbose': -1}), lgb.Dataset(val_x, label=val_y, params={'verbose': -1})
# зафиксируем параметры LightGBM, которые мы оптимизировать не хотим
# параметры LightGBM https://lightgbm.readthedocs.io/en/latest/Parameters.html?highlight=verbosity#learning-control-parameters
params = {
          'objective': 'binary',
          'metric': 'binary_logloss',
          'verbosity': -1,
          'verbose': -1,
          'boosting_type': 'gbdt',
}
# инициализируем модель LightGBM | параметры lgb.train https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.train.html
# все вычисления, которые видим в output, происходят здесь
model = lgb.train(params,
                  dtrain,
                  valid_sets = [dtrain, dval],
                  verbose_eval = False,
                  early_stopping_rounds = 100)

[I 2020-12-01 18:20:34,341] A new study created in memory with name: no-name-60936a11-f9df-439d-8e8d-82c1a60d62b7
min_data_in_leaf, val_score: 0.115758: 100%|##########| 5/5 [00:00<00:00,  9.97it/s]


In [ ]:
# получим предсказания модели
# np.rint: округление до ближайшего целого числа
prediction = np.rint(model.predict(val_x, num_iteration=model.best_iteration))
# измерим долю правильных ответов
accuracy = accuracy_score(val_y, prediction)
accuracy

0.972027972027972

Посмотрим лучшие параметры

In [ ]:
best_params = model.params
print("Best params:", best_params)
print("  Accuracy = {}".format(accuracy))
print("  Params: ")
for key, value in best_params.items():
    print("    {}: {}".format(key, value))

Best params: {'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1, 'verbose': -1, 'boosting_type': 'gbdt', 'feature_pre_filter': False, 'lambda_l1': 2.099942822514516e-07, 'lambda_l2': 1.4690619001671335e-06, 'num_leaves': 31, 'feature_fraction': 0.4, 'bagging_fraction': 0.6069644374821755, 'bagging_freq': 4, 'min_child_samples': 50}
  Accuracy = 0.972027972027972
  Params: 
    objective: binary
    metric: binary_logloss
    verbosity: -1
    verbose: -1
    boosting_type: gbdt
    feature_pre_filter: False
    lambda_l1: 2.099942822514516e-07
    lambda_l2: 1.4690619001671335e-06
    num_leaves: 31
    feature_fraction: 0.4
    bagging_fraction: 0.6069644374821755
    bagging_freq: 4
    min_child_samples: 50


### Дополнительные материалы

Туториал по Optuna: https://www.kaggle.com/corochann/optuna-tutorial-for-hyperparameter-optimization

Оптимизация Optuna + PyTorch: https://towardsdatascience.com/https-medium-com-perlitz-hyper-parameter-optimization-with-optuna-1c32785e7df

Примеры реализаций: https://www.kaggle.com/search?q=optuna

Библиотеки для оптимизации: https://docs.ray.io/en/master/tune/api_docs/suggestion.html#tune-search-alg

Интеграция с Light: https://medium.com/optuna/lightgbm-tuner-new-optuna-integration-for-hyperparameter-optimization-8b7095e99258

Обзор Optuna 2.0: https://medium.com/optuna/optuna-v2-3165e3f1fc2

Объяснение работы алгоритма Hyperband для Pruning: https://medium.com/optuna/optuna-supports-hyperband-93b0cae1a137

Сравнение алгоритмов оптимизации SHA и ASHA: https://blog.ml.cmu.edu/2018/12/12/massively-parallel-hyperparameter-optimization/

Ссылка на библиотеку Optuna: https://optuna.readthedocs.io/en/stable/reference/index.html

Документация LightGBM для Python: https://lightgbm.readthedocs.io/en/latest/Python-Intro.html

Параметры LightGBM: https://lightgbm.readthedocs.io/en/latest/Parameters.html?highlight=verbosity#learning-control-parameters

Параметры LightGBM train: https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.train.html

Применение sklearn: https://towardsdatascience.com/how-to-make-your-model-awesome-with-optuna-b56d490368af